In [ ]:
#!pip install pypdf
#!pip install python-dotenv

In [ ]:
#!pip -q install git+https://github.com/huggingface/transformers

In [ ]:
#!pip install -q datasets loralib sentencepiece

In [ ]:
#!pip install -q einops accelerate langchain bitsandbytes

In [ ]:
#!pip install sentence_transformers

In [ ]:

#!pip install llama-index

In [ ]:
#pip install -U flash-attn --no-build-isolation

In [ ]:

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import HuggingFaceLLM

In [ ]:
documents = SimpleDirectoryReader("example_data").load_data()

In [ ]:
len(documents)

In [ ]:

#!huggingface-cli login


In [ ]:
from llama_index.prompts.prompts import SimpleInputPrompt
system_prompt = "You are a Q&A assistant. Your goal is to answer questions as accurately as possible based on the instructions and context provided."
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = "<|USER|>{query_str}<|ASSISTANT|>"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "auto" # the device to load the model onto


llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.1, "do_sample": True},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map=device,
    tokenizer_kwargs={"max_length": 4096},
    model_kwargs={"torch_dtype": torch.float32},

)
     
#llm = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-v0.1", torch_dtype=torch.float16)
#tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-v0.1")

#prompt = "My favourite condiment is"

#model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
#model.to(device)

#generated_ids = model.generate(**model_inputs, max_new_tokens=100, do_sample=True)
#tokenizer.batch_decode(generated_ids)[0]

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
embed_model =HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2",model_kwargs = {'device': 'cpu'}
)

In [ ]:
service_context = ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

In [ ]:
index = VectorStoreIndex.from_documents(documents, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
response = query_engine.query("what was the revenuew of Q3 of BAyer?")
print(response)